# Домашнее задание к лекции "Статистика. Практика"

Регина, добрый день!

Отличная работа, гипотезы сформулированы и протестированы верно. Замечательно справились с моделью :)

Для дальнейшего развития рекомендую вам изучить, как можно настраивать параметры модели , чтобы результаты были лучше. Для начала можно почитать вот эту статью: https://medium.com/@annabiancajones/sentiment-analysis-on-reviews-feature-extraction-and-logistic-regression-43a29635cc81

В т.ч. можно оптимизировать модель под другую метрику качества (а не accuracy) при помощи функций GridSearchCV или RandomSearchCV, чтобы достигать цели уменьшения именно тех ошибок, которые она совершает больше всего.

# Задание 1
Вернемся к набору данных о видеоиграх.

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?
2) Критикам нравятся больше игры на PC или на PS4?
3) Критикам больше нравятся стрелялки или стратегии?

Для каждого вопроса:

сформулируйте нулевую и альтернативную гипотезы;
выберите пороговый уровень статистической значимости;
опишите полученные результаты статистического теста.

1) Нужно сформулировать нулевую и альтернативную гипотезы;
2) Определить пороговый уровень статистической значимости;
3) Сколько у нас выборок (какой вариант теста используем)?
3) Нужен ли односторонний, либо двухсторонний вариант теста?
5) Определить, равна ли дисперсия групп.

# 1) Как критики относятся к спортивным играм?

In [1]:
import pandas as pd
import scipy.stats as stats

In [2]:
data = pd.read_csv('video_games_sales.csv')
data.head()

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,...,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,...,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,...,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,...,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,...,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,...,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


In [3]:
data_1 = data[['Critic_Score', 'Genre']]
data_1.dropna()

,Critic_Score,Genre
0,7.7,Sports
1,10.0,Platform
2,8.2,Racing
4,8.0,Sports
5,9.4,Role-Playing
...,...,...
55387,5.4,Adventure
55423,9.0,Music
55490,8.0,Action
55528,7.0,Action-Adventure


In [4]:
data_1.describe()

,Critic_Score
count,6536.000000
mean,7.213709
std,1.454079
min,1.000000
25%,6.400000
50%,7.500000
75%,8.300000
max,10.000000


In [5]:
data_sport = data_1[['Critic_Score']].loc[data_1.Genre == 'Sports']
#с жанром спорт
data_no_sport = data_1[['Critic_Score']].loc[data_1.Genre != 'Sports']
#все остальные

In [6]:
alpha = 0.05
#Применяем двухсторонний двухвыборочный критерий Стьюдента
result = stats.ttest_ind(data_sport['Critic_Score'], 
                         data_no_sport['Critic_Score'], equal_var=False)
#ttest_ind для двух независимых выборок  
print(result)
if (result.pvalue < alpha):
    print('Отвергаем H0, Средняя оценка критиками жанра Sports больше, чем средняя оценка критиками другие жанров')
else:
    print('Не отвергаем H0')

Ttest_indResult(statistic=nan, pvalue=nan)
Не отвергаем H0


C:\Users\Hastya\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Не отвергаем H0, потому что statistic>0, значит средняя оценка жанра Sports больше, чем средняя оценка критиками другие жанров.
pvalue>5% разница есть, но она для нас не значима и мы не отвергаем Н0.

# 2) Критикам нравятся больше игры на PC или на PS4?

In [7]:
data_PC = data[['Critic_Score']].loc[data.Platform == 'PC'].dropna()
#с жанром спорт
data_PS4 = data[['Critic_Score']].loc[data.Platform == 'PS4'].dropna()

In [8]:
alpha = 0.05
#Применяем двухсторонний двухвыборочный критерий Стьюдента
result = stats.ttest_ind(data_PC['Critic_Score'], 
                         data_PS4['Critic_Score'], equal_var=False)
#ttest_ind для двух независимых выборок  
print(result)
if (result.pvalue < alpha):
    print('Отвергаем Н0 средние оценки критиков отличаются')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-2.7394476056351627, pvalue=0.006931808250254211)
Отвергаем Н0 средние оценки критиков отличаются


In [9]:
import numpy as np
print(np.mean(data_PC))
print(np.mean(data_PS4))

Critic_Score    7.54185
dtype: float64
Critic_Score    7.904587
dtype: float64


Мы отвергаем Н0 о равенстве средних оценок. И средняя оценка для PC<PS4, значит критикам нравится больше PS4

# 3) Критикам больше нравятся стрелялки или стратегии?

In [10]:
data_Sh = data[['Critic_Score']].loc[data.Genre == 'Shooter'].dropna()
#с жанром спорт
data_St = data[['Critic_Score']].loc[data.Genre == 'Strategy'].dropna()

In [11]:
alpha = 0.05
#Применяем двухсторонний двухвыборочный критерий Стьюдента
result = stats.ttest_ind(data_Sh['Critic_Score'], 
                         data_St['Critic_Score'], equal_var=False)
#ttest_ind для двух независимых выборок  
print(result)
if (result.pvalue < alpha):
    print('Отвергаем Н0 средние оценки критиков отличаются')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-1.6073949711166526, pvalue=0.10838786414223071)
Не отвергаем нулевую гипотезу


In [12]:
print(np.mean(data_Sh))
print(np.mean(data_St))

Critic_Score    7.286833
dtype: float64
Critic_Score    7.429268
dtype: float64


statistic<0, значит средние оценки стрелялок, ниже чем средние оценки стратегий.
pvalue=0.10 > 0.05 Разница средних статистически незначима. Средняя оценка выше у Strategy. 

# Задание 2


Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

1) Привидите весь текст к нижнему регистру;
2) Удалите мусорные символы;
3) Удалите стоп-слова;
4) Привидите все слова к нормальной форме;
5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=cname)
Можете поэкспериментировать с параметрами TfidfVectorizer;
6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;
7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;
8) Опишите результаты при помощи confusion_matrix;
9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [13]:
# импорт библиотек
import pandas as pd
import numpy as np
from scipy import stats as st
from scipy.stats import mannwhitneyu
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [14]:
df_spam= pd.read_csv('spam.csv')
df_spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
#1) Привидите весь текст к нижнему регистру;
#2) Удалите мусорные символы;
#3) Удалите стоп-слова;
#4) Привидите все слова к нормальной форме;

def message_lower(row):
  
    lower_case = row.lower()
    words = re.sub(r'[\W_]+',' ', lower_case)   
    splitted = words.split() # Разбиваем текст  
    stopwords_set = set(stopwords.words('english'))
    without_sw = [word for word in splitted if word not in stopwords_set]
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_text = [wordnet_lemmatizer.lemmatize(word) for word in without_sw]
    
    message = ' '.join(map(str, lemmatized_text))
    
    return message


In [16]:
df_spam['Message'] = df_spam['Message'].apply(message_lower)
df_spam

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though
...,...,...
5567,spam,2nd time tried 2 contact u u 750 pound prize 2...
5568,ham,ü b going esplanade fr home
5569,ham,pity mood suggestion
5570,ham,guy bitching acted like interested buying some...


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_spam.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=None)
#Можете поэкспериментировать с параметрами TfidfVectorizer;
tfidf_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,8038,8039,8040,8041,8042,8043,8044,8045,8046,8047
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
#6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_spam['Category'], test_size=0.3, random_state=42)

In [55]:
len(y_test)

1672

In [56]:
#7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;
lr = LogisticRegression(random_state=42)

In [58]:
lr.fit(X_train, y_train)
#обучение

LogisticRegression(random_state=42)

In [59]:
# делаем прогноз на тестовой выборке
lr.predict(X_test)

array(['ham', 'ham', 'ham', ..., 'ham', 'spam', 'ham'], dtype=object)

In [60]:
# Оценим ее точность на тестовых данных
from sklearn.metrics import accuracy_score
accuracy_score(y_test, lr.predict(X_test))

0.958732057416268

In [63]:
#8) Опишите результаты при помощи confusion_matrix;
print(confusion_matrix(y_test, lr.predict(X_test), labels=['ham', 'spam']))

[[1445    3]
 [  66  158]]


In [65]:
#66 определил спам и 3 ложных 

9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [68]:
df_spam['Fact'] = pd.DataFrame(y_test)
df_spam['Prediction'] = pd.DataFrame(lr.predict(X_test))
result = df_spam[df_spam.Fact != df_spam.Prediction]
result.drop(['Category'], axis=1).dropna()

,Message,Fact,Prediction
8,winner valued network customer selected receiv...,spam,ham
12,urgent 1 week free membership 100 000 prize ja...,spam,ham
15,xxxmobilemovieclub use credit click wap link n...,spam,ham
17,eh u remember 2 spell name yes v naughty make ...,ham,spam
19,england v macedonia dont miss goal team news t...,spam,ham
...,...,...,...
1544,hello orange 1 month free access game news spo...,spam,ham
1564,get ready put excellent sub face,ham,spam
1583,yep derek house see sunday lt 3,ham,spam
1598,urgent mobile number awarded 2000 prize guaran...,spam,ham
